In [38]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import math
from scipy.stats import wasserstein_distance

In [39]:
# Local on Windows 10 box
WD = os.path.join("E:\\", "BUSTEDS-MH")

tag = "empirical_unmasked_selectome"

# Additional declares
BUSTEDS_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS")
BUSTEDS_MH_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS-MH")

OUTPUT_CSV = os.path.join(WD, "tables", "Table_" + tag.upper() + "_BUSTEDS_and_BUSTEDS-MH.csv")

ER_Threshold = 5

In [40]:
def read_json(filename):
    #print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        print("# -- Error -- file is empty:", filename)
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)

In [41]:
BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".json")]
BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]

print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))


# Number of BUSTEDS results: 13307
# Number of BUSTEDS-MH results: 13299


## Look over BUSTEDS-MH Files

In [42]:
df_dict = {}

for item in tqdm(BUSTEDS_MH_DIR_FILES):
    basename = os.path.basename(item).replace(".nex.BUSTEDS-MH.json", "")
    # Read 
    json_data_BUSTEDS_MH = read_json(item)
    
    if json_data_BUSTEDS_MH == []: continue
    
    #print("# Data loaded")
    df_dict[basename] = {"Method": "BUSTEDS-MH"}
    df_dict[basename].update({"Sequences": json_data_BUSTEDS_MH["input"]["number of sequences"]})
    df_dict[basename].update({"Codons": json_data_BUSTEDS_MH["input"]["number of sites"]})
    df_dict[basename].update({"LRT p-value": json_data_BUSTEDS_MH["test results"]["p-value"]})

    # cAIC
    df_dict[basename].update({"cAIC": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["AIC-c"]})
    
    # CV of omega
    #A = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    #B = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    #C = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    #df_dict[basename].update({"CV(omega)": cv([A, B, C])})
    
    # CV of alpha
    #D = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    #E = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    #F = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    #df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
    
    #Omegas and proportions
    data = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
    w1 = round(data["0"]["omega"], 4)
    p1 = round(data["0"]["proportion"], 4)
    w2 = round(data["1"]["omega"], 4)
    p2 = round(data["1"]["proportion"], 4)
    w3 = round(data["2"]["omega"], 4)
    p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"w1": w1, "p1": p1})
    df_dict[basename].update({"w2": w2, "p2": p2})
    df_dict[basename].update({"w3": w3, "p3": p3})
    
    df_dict[basename].update({"CV(omega)": cv([w1, w2, w3])})
    
    # SRV rates and proportions
    data = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    s1 = round(data["0"]["rate"], 4)
    s_p1 = round(data["0"]["proportion"], 4)
    s2 = round(data["1"]["rate"], 4)
    s_p2 = round(data["1"]["proportion"], 4)
    s3 = round(data["2"]["rate"], 4)
    s_p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
    df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
    df_dict[basename].update({"SRV3": s3, "SRV_p3": s3})
    df_dict[basename].update({"CV(alpha)": cv([s1, s2, s3])})
    
    # DH rate, TH rate, TH_SI rate
    df_dict[basename].update({"DH_Rate": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 2 nucleotides are changed instantly within a single codon"])})
    df_dict[basename].update({"TH_Rate": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon"])})
    df_dict[basename].update({"TH_Rate_SI": float(json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"])})

    # ER Sites, thresholded
    ER_SITES = []
    ER_df_dict = {}
    if "constrained" in json_data_BUSTEDS_MH["Evidence Ratios"].keys():
        #print("# ER Constrained Sites:", len(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES.append(str(site + 1))
                ER_df_dict[site + 1] = {"BUSTEDS-MH ER": val}
            #end if
        #end for
        # add assert that there are more than 0 sites here.
        df_dict[basename].update({"BUSTEDS-MH_num_ER_Sites":  len(ER_df_dict.keys())})
        x = ER_df_dict.keys()
        x = [str(x) for x in x]
        df_dict[basename].update({"BUSTEDS-MH_ER_Sites":  "|".join(x)})
        #print(ER_df_dict.keys())
    #end if 
# end for

df_MH = pd.DataFrame.from_dict(df_dict, orient="index")
df_MH = df_MH.reset_index()
df_MH.index += 1
df_MH.rename(columns={'index': 'Gene'}, inplace = True)
#df_MH

100%|████████████████████████████████████████████████████████████████████████████| 13299/13299 [02:15<00:00, 98.27it/s]


## Look over BUSTEDS Files

In [43]:
df_dict = {}

for item in tqdm(BUSTEDS_DIR_FILES):
    basename = os.path.basename(item).replace(".nex.BUSTEDS.json", "")
    # Read json
    #print()
    json_data_BUSTEDS = read_json(item)
    #print("# Data loaded:", item)
    
    df_dict[basename] = {"Method": "BUSTEDS"}
    df_dict[basename].update({"Sequences": json_data_BUSTEDS["input"]["number of sequences"]})
    df_dict[basename].update({"Codons": json_data_BUSTEDS["input"]["number of sites"]})
    df_dict[basename].update({"LRT p-value": json_data_BUSTEDS["test results"]["p-value"]})

    # cAIC
    df_dict[basename].update({"cAIC": json_data_BUSTEDS["fits"]["Unconstrained model"]["AIC-c"]})
    
    A = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    df_dict[basename].update({"CV(omega)": cv([A, B, C])})
    
    D = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
    
    #Omegas and proportions
    data = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
    w1 = round(data["0"]["omega"], 4)
    p1 = round(data["0"]["proportion"], 4)
    w2 = round(data["1"]["omega"], 4)
    p2 = round(data["1"]["proportion"], 4)
    w3 = round(data["2"]["omega"], 4)
    p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"w1": w1, "p1": p1})
    df_dict[basename].update({"w2": w2, "p2": p2})
    df_dict[basename].update({"w3": w3, "p3": p3})
    
    # SRV rates and proportions
    data = json_data_BUSTEDS["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    s1 = round(data["0"]["rate"], 4)
    s_p1 = round(data["0"]["proportion"], 4)
    s2 = round(data["1"]["rate"], 4)
    s_p2 = round(data["1"]["proportion"], 4)
    s3 = round(data["2"]["rate"], 4)
    s_p3 = round(data["2"]["proportion"], 4)
    df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
    df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
    df_dict[basename].update({"SRV3": s3, "SRV_p3": s3})
    
    # ER Sites
    ER_SITES = []
    ER_df_dict = {}
    
    if "constrained" in json_data_BUSTEDS["Evidence Ratios"].keys():
        #print("# ER Constrained Sites:", len(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]))
        for site, val in enumerate(json_data_BUSTEDS["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES.append(str(site + 1))
                ER_df_dict[site + 1] = {"BUSTEDS ER": val}
            #end if
        #end for
        df_dict[basename].update({"num_ER_Sites":  int(len(ER_df_dict.keys()))})
        df_dict[basename].update({"BUSTEDS_num_ER_Sites":  len(ER_df_dict.keys())})
        x = ER_df_dict.keys()
        x = [str(x) for x in x]
        df_dict[basename].update({"BUSTEDS_ER_Sites":  "|".join(x)})
        #print(ER_df_dict.keys())
    #end if   
    

# End for

df = pd.DataFrame.from_dict(df_dict, orient="index")
df = df.reset_index()
df.index += 1
df.rename(columns={'index': 'Gene'}, inplace = True)
#df

100%|████████████████████████████████████████████████████████████████████████████| 13307/13307 [02:32<00:00, 87.46it/s]


## Calculate cAIC statistics

In [48]:
df["ΔcAIC"] = ""
df["RelativeSupport"] = ""
df["ER_Sites_Intersection"] = ""

for index, row in tqdm(df_MH.iterrows()):
    #MH_cAIC = df_MH[]
    #df_temp = df[df["Gene"] == gene]
    #print(df_temp)
    #print(row["Gene"], row["cAIC"])    
    gene = row["Gene"]
    MH_cAIC = float(row["cAIC"])
    
    BUSTEDS_row = df[df["Gene"] == row["Gene"]]
    index_BUSTEDS = df[df["Gene"] == row["Gene"]].index
    
    try:
        BUSTEDS_cAIC = float(BUSTEDS_row["cAIC"])
    except:
        print(BUSTEDS_row)
        pass
    #end try
    
    BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3 = float(row["w1"]), float(row["w2"]), float(row["w3"])
    try:
        BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3 = float(BUSTEDS_row["w1"]), float(BUSTEDS_row["w2"]), float(BUSTEDS_row["w3"])
    except:
        print(BUSTEDS_row["w1"], BUSTEDS_row["w2"], BUSTEDS_row["w3"])
        continue
    #end try
    
    BUSTEDSMH_p1, BUSTEDSMH_p2, BUSTEDSMH_p3 = float(row["p1"]), float(row["p2"]), float(row["p3"])
    BUSTEDS_p1, BUSTEDS_p2, BUSTEDS_p3 = float(BUSTEDS_row["p1"]), float(BUSTEDS_row["p2"]), float(BUSTEDS_row["p3"])
    
    BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3 = float(row["SRV1"]), float(row["SRV2"]), float(row["SRV3"])
    BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3 = float(BUSTEDS_row["SRV1"]), float(BUSTEDS_row["SRV2"]), float(BUSTEDS_row["SRV3"])
    
    BUSTEDSMH_sp1, BUSTEDSMH_sp2, BUSTEDSMH_sp3 = float(row["SRV_p1"]), float(row["SRV_p2"]), float(row["SRV_p3"])
    BUSTEDS_sp1, BUSTEDS_sp2, BUSTEDS_sp3 = float(BUSTEDS_row["SRV_p1"]), float(BUSTEDS_row["SRV_p2"]), float(BUSTEDS_row["SRV_p3"])
    
    #print(float(BUSTEDS_cAIC["cAIC"]))
    best_model = min(MH_cAIC, BUSTEDS_cAIC)
    #print()
    #print("# Gene:", row["Gene"])

    if BUSTEDS_cAIC == best_model:
        which_is_best = "BUSTEDS"
        delta_cAIC = MH_cAIC - best_model
        relative_support = math.exp(-delta_cAIC/2)
        # add to table
        #df.at['C', 'x'] = 10
        # 
        df.at[index_BUSTEDS, "ΔcAIC"] = delta_cAIC
        df.at[index_BUSTEDS, "RelativeSupport"] = relative_support
    elif MH_cAIC == best_model:
        which_is_best = "BUSTEDS-MH"
        delta_cAIC = BUSTEDS_cAIC - best_model
        relative_support = math.exp(-delta_cAIC/2)
        df_MH.at[index, "ΔcAIC"] = delta_cAIC
        df_MH.at[index, "RelativeSupport"] = relative_support
    else:
        pass
    #end if
    #print("# Best model is:", best_model, which_is_best, "by", delta_cAIC)
    #print("# With relative support:", relative_support)
    
    # Intersections of ER Sites.
    #print("# Examining ER Sites")
    # BUSTEDS-MH_ER_Sites
    # BUSTEDS_ER_Sites
    try:
        BUSTEDS_MH_ER_Sites = row["BUSTEDS-MH_ER_Sites"].split("|")
        BUSTEDS_df = df[df["Gene"] == row["Gene"]]
        BUSTEDS_ER_Sites    = BUSTEDS_df["BUSTEDS_ER_Sites"].tolist()[0].split("|")
        #print(BUSTEDS_MH_ER_Sites, BUSTEDS_ER_Sites)
        intersection = set(BUSTEDS_MH_ER_Sites).intersection(BUSTEDS_ER_Sites)
        #print(intersection)
        df.at[index, "ER_Sites_Intersection"] = "|".join(intersection)
    except:
        #print("ERROR --", row["BUSTEDS-MH_ER_Sites"])
        pass
    #end try
    
    
    # Wasserstein Distance (Earth Movers)
    #print("Omegas:", BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3, BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3)
    WD_unweighted_omega = wasserstein_distance([BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3],
                                               [BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3])
    
    WD_weighted_omega   = wasserstein_distance([BUSTEDSMH_w1, BUSTEDSMH_w2, BUSTEDSMH_w3], 
                                               [BUSTEDS_w1, BUSTEDS_w2, BUSTEDS_w3],
                                               [BUSTEDSMH_p1, BUSTEDSMH_p2, BUSTEDSMH_p3],
                                               [BUSTEDS_p1, BUSTEDS_p2, BUSTEDS_p3])
    
    #print("SRV:", BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3, BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3)
    
    WD_unweighted_srv   = wasserstein_distance([BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3], 
                                               [BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3])
    
    WD_weighted_srv     = wasserstein_distance([BUSTEDSMH_s1, BUSTEDSMH_s2, BUSTEDSMH_s3], 
                                               [BUSTEDS_s1, BUSTEDS_s2, BUSTEDS_s3], 
                                               [BUSTEDSMH_sp1, BUSTEDSMH_sp2, BUSTEDSMH_sp3], 
                                               [BUSTEDS_sp1, BUSTEDS_sp2, BUSTEDS_sp3])
    
    df_MH.at[index, "WD_unweighted(omega)"] = WD_unweighted_omega
    df_MH.at[index, "WD_weighted(omega)"] = WD_weighted_omega
    df_MH.at[index, "WD_unweighted(srv)"] = WD_unweighted_srv
    df_MH.at[index, "WD_weighted(srv)"] = WD_weighted_srv
    
    
#end for
    

10781it [00:45, 220.15it/s]

Empty DataFrame
Columns: [Gene, Method, Sequences, Codons, LRT p-value, cAIC, CV(omega), CV(alpha), w1, p1, w2, p2, w3, p3, SRV1, SRV_p1, SRV2, SRV_p2, SRV3, SRV_p3, num_ER_Sites, BUSTEDS_num_ER_Sites, BUSTEDS_ER_Sites, ΔcAIC, RelativeSupport, ER_Sites_Intersection]
Index: []

[0 rows x 26 columns]
Series([], Name: w1, dtype: float64) Series([], Name: w2, dtype: float64) Series([], Name: w3, dtype: float64)


13299it [00:55, 238.97it/s]


## Concat tables


In [49]:
#df = df.sort_values(by="Sequences", ascending=False)
#df_MH = df_MH.sort_values(by="Sequences", ascending=False)

result = pd.concat([df_MH, df])
result = result.fillna("")
result = result.sort_values(by=["Gene", "Method"], ascending=True)
result = result.reset_index(drop=True)
result.index += 1
result

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,w1,p1,w2,p2,...,ΔcAIC,RelativeSupport,WD_unweighted(omega),WD_weighted(omega),WD_unweighted(srv),WD_weighted(srv),num_ER_Sites,BUSTEDS_num_ER_Sites,BUSTEDS_ER_Sites,ER_Sites_Intersection
1,ENSGT00390000000002.Euteleostomi.001,BUSTEDS,24,428,0.500000,15230.383413,0.0083,0.8481,0.2237,0.1472,...,,,,,,,0.0,0.0,,
2,ENSGT00390000000002.Euteleostomi.001,BUSTEDS-MH,24,428,0.453178,15218.395700,0.0108,0.8748,0.2630,0.1247,...,11.987712,0.002494,1.72,0.018824,0.002633,0.004728,,,,
3,ENSGT00390000000002.Euteleostomi.002,BUSTEDS,35,520,0.500000,28163.498555,0.0171,0.9297,0.2579,0.0703,...,4.460285,0.107513,,,,,,,,
4,ENSGT00390000000002.Euteleostomi.002,BUSTEDS-MH,35,520,0.500000,28167.958840,0.0288,1.0000,0.0487,0.0000,...,,,0.114767,0.026983,0.012767,0.010951,,,,
5,ENSGT00390000000002.Euteleostomi.003,BUSTEDS,30,381,0.342345,18555.324235,0.0113,0.4917,0.0967,0.4930,...,,,,,,,0.0,0.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26602,ENSGT00680000101373.Euteleostomi.001,BUSTEDS-MH,6,438,0.500000,3894.546024,0.0363,0.0000,0.3503,1.0000,...,,,0.000967,0.0001,0.1055,0.078059,,,,
26603,ENSGT00680000101376.Euteleostomi.001,BUSTEDS,6,344,0.500000,2828.089034,0.0000,0.8940,0.0000,0.1060,...,6.190516,0.045263,,,,,,,,
26604,ENSGT00680000101376.Euteleostomi.001,BUSTEDS-MH,6,344,0.500000,2834.279550,0.0000,1.0000,0.0006,0.0000,...,,,0.0038,0.0,0.000567,0.001793,,,,
26605,ENSGT00680000101377.Euteleostomi.001,BUSTEDS,6,405,0.015805,4896.192723,0.1444,0.9866,0.1454,0.0032,...,,,,,,,1.0,1.0,371,


In [50]:
"""dfv = result
dfv = dfv[['Gene', 'Sequences', 'Method', 'Codons', 'LRT p-value', 'cAIC', 'delta cAIC (best model)', 'Relative support',
       'CV(omega)', 'CV(alpha)', 'omega_3', 'proportion_3', 'DH_Rate',
       'TH_Rate', 'TH_Rate_SI', 'num_ER_Sites']]

dfv = dfv.fillna("")
dfv = dfv.sort_values(by=["Gene", "Method"], ascending=True)
dfv = dfv.reset_index(drop=True)
dfv.index += 1
"""

#styled_table = result.style.background_gradient()
#styled_table

'dfv = result\ndfv = dfv[[\'Gene\', \'Sequences\', \'Method\', \'Codons\', \'LRT p-value\', \'cAIC\', \'delta cAIC (best model)\', \'Relative support\',\n       \'CV(omega)\', \'CV(alpha)\', \'omega_3\', \'proportion_3\', \'DH_Rate\',\n       \'TH_Rate\', \'TH_Rate_SI\', \'num_ER_Sites\']]\n\ndfv = dfv.fillna("")\ndfv = dfv.sort_values(by=["Gene", "Method"], ascending=True)\ndfv = dfv.reset_index(drop=True)\ndfv.index += 1\n'

## Save table

In [52]:
print("Saving results to:", OUTPUT_CSV)
result.to_csv(OUTPUT_CSV, index=False)

Saving results to: E:\BUSTEDS-MH\tables\Table_EMPIRICAL_UNMASKED_SELECTOME_BUSTEDS_and_BUSTEDS-MH.csv


## End of file

In [12]:
# Note Negative delta LL are convergence problems

In [13]:
# Lower AIC values indicate a better-fit model, and a model with a delta-AIC (the difference between the two AIC values being compared) of more than -2 is considered significantly better than the model it is being compared to

In [14]:
# Earth Mover's (Kantorovich) distance between two distrbuitions if you want a single number